In [1]:
import pandas as pd
import gzip

def clinvar_vcf_to_pd(vcf_path: str) -> list:
    """
    From a vcf file it creates a list containing information of each mutation
    described in clinvar as a dictionary.
    Args:
        vcf_path (str): path to the vcf file where we will extract the information
    Returns:
        variants_params (list): list of dictionaries where each item of a list is a dictionary
            that describes the variant
    """
    
    # list where we will store dictionary params of each variant
    variants_params = list()
    with gzip.open(vcf_path, "rt") as file:
        for line in file:
            # remove \n
            line = line.strip()
            # dictinary where we will store each parameter of the variant
            if line.startswith("#"):
                # descriptor lines not interested in
                continue
            fields = line.split("\t")
            # dictinary where we will store each parameter of the variant
            # obtaining parameters from each variant 
            chrom = fields[0]
            pos = fields[1]
            id = fields[2]
            ref = fields[3]
            alt = fields[4]
            qual = fields[5]
            filter = fields[6]
            info = fields[7]
            dict_params = {
                "Chrom" : chrom,
                "Pos" : pos,
                "Id" : id,
                "Ref" : ref,
                "Alt" : alt,
                "Qual" : qual,
                "Filter" : filter,

            }
            # print(fields)

            # in info we have different parameters 
            clnv_params = info.split(";")

            for clnv_param in clnv_params:
                key_value = clnv_param.split("=")
                key = key_value[0]
                value = key_value[1]

                # add each annotation as a key_dict value in dict_params    
                dict_params[key] = value
            
            # create a list of  mutations with dicts that store its information
            variants_params.append(dict_params)
                
    return(variants_params)


def clean_clinvar_file(clinvar_path, cleaned_file):
    
    # we just want mutations that are classified as the following items
    sig = ["Pathogenic", "Benign", "Likely_pathogenic", "Likely_benign", "Uncertain_significance"]
    
    with gzip.open(clinvar_path, "rt", encoding='utf-8') as file:
        with open(cleaned_file, "w") as write_file:
            for line in file:
                # dictinary where we will store each parameter of the variant
                if line.startswith("#"):
                    write_file.write(line)

                    # descriptor lines not interested in
                    continue
                # It is a tab seperated file so we split it using tabs
                fields = line.split("\t")
                # all annotations are in info field which is positioned at 7
                info = fields[7]

                # clinvar annotations are seperated by ;
                clnv_params = info.split(";")

                for clnv_param in clnv_params:
                    # each annotation is a key=value pair
                    key_value = clnv_param.split("=")
                    key = key_value[0]
                    value = key_value[1]
                    # in CLNSIG is the pathogenicity of the mutation
                    if key == "CLNSIG":
                        # we only take main 
                        if any(sign in value for sign in sig):
                            clin_sig = True
                if clin_sig:
                    write_file.write(line)
# clean_clinvar_file("/home/ocanal/Desktop/clinvar/hg19/clinvar_20231104.vcf.gz", "/home/ocanal/Desktop/clinvar/hg19/clinvar_parsed_hg19.vcf")
                    

variants_params = clinvar_vcf_to_pd("/home/ocanal/Desktop/clinvar/hg19/clinvar_20231104.vcf.gz")
df = pd.DataFrame(variants_params)
# print(variants_params)

In [5]:
df

,Chrom,Pos,Id,Ref,Alt,Qual,Filter,ALLELEID,CLNDISDB,CLNDN,...,RS,AF_EXAC,AF_ESP,CLNSIGCONF,AF_TGP,CLNVI,CLNDISDBINCL,CLNDNINCL,CLNSIGINCL,DBVARID
0,1,69134,2205837,A,G,.,.,2193183,"MeSH:D030342,MedGen:C0950123",Inborn_genetic_diseases,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,69581,2252161,C,G,.,.,2238986,"MeSH:D030342,MedGen:C0950123",Inborn_genetic_diseases,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,69682,2396347,G,A,.,.,2386655,"MeSH:D030342,MedGen:C0950123",Inborn_genetic_diseases,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,69769,2288999,T,C,.,.,2278803,"MeSH:D030342,MedGen:C0950123",Inborn_genetic_diseases,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,69995,2351346,G,C,.,.,2333177,"MeSH:D030342,MedGen:C0950123",Inborn_genetic_diseases,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300570,NW_003315925.1,83614,17735,TC,T,.,.,32774,"Human_Phenotype_Ontology:HP:0032224,MedGen:C00...",ABO_blood_group_system|not_provided,...,1556058284,NaN,NaN,NaN,NaN,OMIM:110300.0001,NaN,NaN,NaN,NaN
2300571,NW_003315947.1,181683,156304,C,G,.,.,166084,MedGen:C3272265|MedGen:CN517202|.,Three_Vessel_Coronary_Disease|not_provided|PLA...,...,281865545,NaN,NaN,NaN,NaN,ClinGen:CA123258|OMIM:173445.0001,NaN,NaN,NaN,NaN
2300572,NW_003315950.2,355765,206,G,A,.,.,15245,"MONDO:MONDO:0013193,MedGen:C2750473,OMIM:61323...","Thyrotoxic_periodic_paralysis,_susceptibility_...",...,672601244,NaN,NaN,NaN,NaN,ClinGen:CA114051|OMIM:613236.0003|UniProtKB:B7...,NaN,NaN,NaN,NaN
2300573,NW_003315950.2,356212,205,C,T,.,.,15244,"MONDO:MONDO:0013193,MedGen:C2750473,OMIM:61323...","Thyrotoxic_periodic_paralysis,_susceptibility_...",...,527236158,NaN,NaN,NaN,NaN,ClinGen:CA114049|OMIM:613236.0002|UniProtKB:B7...,NaN,NaN,NaN,NaN


In [2]:
import matplotlib.pyplot as plt
for column_name in df.columns:
    sum_na = df[column_name].isna().sum()
    print(column_name, sum_na)
df["CLNREVSTAT"].unique()
rows_with_nan = df[df["CLNSIG"].isna()]
rows_with_nan

# nan values because of https://www.ncbi.nlm.nih.gov/clinvar/docs/clinsig/:
# This value may not be submitted. It is used in the file variant_summary.txt.gz in the path  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/. This file reports  '-' in the ClinicalSignificance column for an allele that was submitted to ClinVar only in combination with another allele (e.g.a submission with an interpretation of a haplotype or a compound heterozygote) and was not interpreted explictly.  ClinVar thus has no interpretation specific to that allele.  To find the the interpretation that includes that allele, you can query ClinVar by the AlleleID,


Chrom 0
Pos 0
Id 0
Ref 0
Alt 0
Qual 0
Filter 0
ALLELEID 0
CLNDISDB 719
CLNDN 719
CLNHGVS 19
CLNREVSTAT 0
CLNSIG 719
CLNVC 0
CLNVCSO 0
GENEINFO 453
MC 14169
ORIGIN 3775
RS 1465846
AF_EXAC 1993029
AF_ESP 2131141
CLNSIGCONF 2196618
AF_TGP 2107477
CLNVI 1904506
CLNDISDBINCL 2298887
CLNDNINCL 2298887
CLNSIGINCL 2298887
DBVARID 2300423


,Chrom,Pos,Id,Ref,Alt,Qual,Filter,ALLELEID,CLNDISDB,CLNDN,...,RS,AF_EXAC,AF_ESP,CLNSIGCONF,AF_TGP,CLNVI,CLNDISDBINCL,CLNDNINCL,CLNSIGINCL,DBVARID
4985,1,1455576,1343247,G,GGCCGAGGCCCGGGCGCGC,.,.,1334892,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"MONDO:MONDO:0032931,MedGen:C5394137,OMIM:61881...","Pontocerebellar_hypoplasia,_hypotonia,_and_res...",1343267:Likely_pathogenic,NaN
13987,1,8045031,487089,G,A,.,.,22107,NaN,NaN,...,74315354,0.00002,NaN,NaN,NaN,OMIM:602533.0006|UniProtKB:Q99497#VAR_034801,"MONDO:MONDO:0011658,MedGen:C1853445,OMIM:60632...",Autosomal_recessive_early-onset_Parkinson_dise...,446717:Pathogenic,NaN
15782,1,10035809,982549,G,A,.,.,970649,NaN,NaN,...,1641792929,NaN,NaN,NaN,NaN,NaN,"MONDO:MONDO:0018998,MeSH:D057130,MedGen:C03395...",Leber_congenital_amaurosis,982550:Pathogenic,NaN
41011,1,25611101,635094,G,T,.,.,622837,NaN,NaN,...,199509194,0.00665,0.02480,NaN,0.14217,NaN,.|.,Partial_RhD|altered_RHD_phenotype,634995:not_provided|1185597:Affects,NaN
41017,1,25617206,635095,C,T,.,.,622838,NaN,NaN,...,113982491,0.00665,0.02452,NaN,0.02157,NaN,.|.,Partial_RhD|altered_RHD_phenotype,634995:not_provided|634996:not_provided|118559...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295933,X,153762251,1723203,G,T,.,.,1780260,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"MONDO:MONDO:0010480,MedGen:C2720289,OMIM:30090...","Anemia,_nonspherocytic_hemolytic,_due_to_G6PD_...",1722757:Likely_pathogenic,NaN
2296005,X,153762653,242750,G,A,.,.,38393,NaN,NaN,...,267606836,0.00001,NaN,NaN,NaN,OMIM:305900.0032,"MONDO:MONDO:0010480,MedGen:C2720289,OMIM:30090...","Anemia,_nonspherocytic_hemolytic,_due_to_G6PD_...",10392:Likely_pathogenic,NaN
2296057,X,153763489,1723202,C,A,.,.,1780230,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"MONDO:MONDO:0010480,MedGen:C2720289,OMIM:30090...","Anemia,_nonspherocytic_hemolytic,_due_to_G6PD_...",1722723:Likely_pathogenic,NaN
2296060,X,153763493,1723201,C,A,.,.,1780229,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"MONDO:MONDO:0010480,MedGen:C2720289,OMIM:30090...","Anemia,_nonspherocytic_hemolytic,_due_to_G6PD_...",1722723:Likely_pathogenic,NaN


In [ ]:
plt.hist(df["Ref"])

(array([2273382.,    4704.,    3461.,    3040.,    2859.,    2762.,
           2666.,    2597.,    2564.,    2540.]),
 array([    0. ,  2519.7,  5039.4,  7559.1, 10078.8, 12598.5, 15118.2,
        17637.9, 20157.6, 22677.3, 25197. ]),
 <BarContainer object of 10 artists>)

In [12]:
rows_with_nan = df[df["CLNSIG"].isna()]
rows_with_nan

,Chrom,Pos,Id,Ref,Alt,Qual,Filter,ALLELEID,CLNDISDB,CLNDN,...,AF_EXAC,AF_ESP,CLNSIGCONF,AF_TGP,CLNVI,CLNDISDBINCL,CLNDNINCL,CLNSIGINCL,DBVARID,CLNREVSTAT_stars


In [4]:
df["CLNSIG"].value_counts()

# https://www.ncbi.nlm.nih.gov/clinvar/docs/review_status/

criteria_review = {
    "practice_guideline" : 4,
    "reviewed_by_expert_panel" : 3,
    "criteria_provided,_multiple_submitters,_no_conflicts" : 2,
    "criteria_provided,_conflicting_interpretations" : 1,
    "criteria_provided,_single_submitter": 1,
    "no_assertion_criteria_provided": 0,
    "no_interpretation_for_the_single_variant": 0,
    "no_assertion_provided": 0
}
    
df["CLNREVSTAT_stars"] = df["CLNREVSTAT"].replace(criteria_review)

# Filter just the ones that has 4, 3 or 2 stars
df = df[df["CLNREVSTAT_stars"].isin([4,3,2])]
df["CLNSIG"].value_counts()

def df_to_vcf(df: pd.DataFrame, header: str) -> str:
    """
    From a dataframe, it extracts the fields Chrom, Pos, Id, Ref, Alt, CLNSIG ref_num and alt_num
    to create a VCF file
    args:
        df(pd.DataFrame): pandas dataframe containing previously defined columns
        header(str): header of the vcf file that the output will contain
    Return:
        output_filename(str): vcf file path
    """
    output_filename = "/home/ocanal/Desktop/clinvar/hg19/filtered_vcf_file.vcf"
    
    with open(output_filename, "w") as output_vcf:
        output_vcf.write(header)
        # defining constant values in vcf
        qual = "."
        filter = "."
        for index, row in df.iterrows():
            chrom = str(row["Chrom"])
            pos = str(row["Pos"])
            rs_id = str(row["Id"])
            ref = str(row["Ref"])
            alt = str(row["Alt"])
            clnsig = str(row["CLNSIG"])
            ref_num = str(row["ref_num"])
            alt_num = str(row["alt_num"])
            
    
            inf_field = f"CLNSIG={clnsig};ref_num={ref_num};alt_num={alt_num}"
            
            vcf_line = [chrom, pos, rs_id, ref, alt, qual, filter, inf_field]
    
            vcf_line = "\t".join(vcf_line)
            output_vcf.write(vcf_line + "\n")
        
    return(output_filename)
            
        
        
        

In [4]:
df["CLNSIG"].unique()
count = df['CLNSIG'].value_counts().get('Likely_pathogenic', 0)
count


4284

In [5]:
df_bening = df.loc[df["CLNSIG"] == "Benign"]

df_patho = df.loc[df["CLNSIG"] == "Pathogenic"]
df_uncertain = df.loc[df["CLNSIG"] == "Uncertain_significance"]
all_df = pd.concat([df_bening, df_patho, df_uncertain])
# all_df.to_csv("clinvar.csv", encoding="utf-8", sep="\t")

In [11]:
all_df
all_df["CLNSIG"].value_counts()

CLNSIG
Uncertain_significance    154309
Benign                     35176
Pathogenic                 28418
Name: count, dtype: int64

In [6]:
# data = pd.read_csv("/home/ocanal/Desktop/clinvar_parsed.vcf", sep="\t")
columns_to_keep = ["Chrom","Pos", "Id", "Ref", "RS", "Alt", "CLNSIG"]
known_df = all_df[columns_to_keep]

known_df["Chrom"].unique()
replace_dict = {
    "X": 23, 
    "Y": 24,
    "MT": 25,
    "NW_003315925.1": 26,
    "NW_003315947.1": 27,
    "NT_113889.1" : 28,
    "NT_167222.1": 29
}

known_df["Chrom"] = known_df["Chrom"].replace(replace_dict)

/tmp/ipykernel_92489/1417531837.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["Chrom"] = known_df["Chrom"].replace(replace_dict)


In [7]:
ref_alt_dict = {
    "G" : 1,
    "A": 2,
    "C": 3,
    "T": 4
}

clnsig_dict = {
    "Pathogenic": 0,
    "Uncertain_significance": 1,
    "Benign": 2
}
known_df["ref_num"] = known_df["Ref"].replace(ref_alt_dict)
known_df["alt_num"] = known_df["Alt"].replace(ref_alt_dict)

known_df["cln_sig_num"] = known_df["CLNSIG"].replace(clnsig_dict)
valid_values = {1,2,3,4}
known_df["ref_num"] = known_df["ref_num"].apply(lambda x: x if x in valid_values else -1)
known_df["alt_num"] = known_df["alt_num"].apply(lambda x: x if x in valid_values else -1)
# known_df.to_csv("/home/ocanal/Desktop/clinvar/hg19/clinvar_hg19_csv_last.csv", sep="\t")

/tmp/ipykernel_92489/2816101851.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["ref_num"] = known_df["Ref"].replace(ref_alt_dict)
/tmp/ipykernel_92489/2816101851.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["alt_num"] = known_df["Alt"].replace(ref_alt_dict)
/tmp/ipykernel_92489/2816101851.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [8]:
known_df

,Chrom,Pos,Id,Ref,RS,Alt,CLNSIG,ref_num,alt_num,cln_sig_num
976,1,898613,775485,C,61746776,T,Benign,3,4,2
1144,1,949608,402986,G,1921,A,Benign,1,2,2
1153,1,949654,1166514,A,NaN,G,Benign,2,1,2
1216,1,955563,387476,G,539283387,C,Benign,1,3,2
1233,1,955597,128310,G,115173026,T,Benign,1,4,2
...,...,...,...,...,...,...,...,...,...,...
2300272,25,15117,693828,T,1603225092,C,Uncertain_significance,4,3,1
2300358,25,15437,235525,G,878853058,A,Uncertain_significance,1,2,1
2300369,25,15467,618216,A,1569484723,G,Uncertain_significance,2,1,1
2300400,25,15579,9683,A,207460002,G,Uncertain_significance,2,1,1


In [12]:
all_df
df_to_vcf(known_df, "#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n")

'/home/ocanal/Desktop/clinvar/hg19/filtered_vcf_file.vcf'

/tmp/ipykernel_70875/2731417348.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["Chrom"] = known_df["Chrom"].replace(replace_dict)


/tmp/ipykernel_70875/2422956088.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["ref_num"] = known_df["Ref"].replace(ref_alt_dict)
/tmp/ipykernel_70875/2422956088.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_df["alt_num"] = known_df["Alt"].replace(ref_alt_dict)


KeyError: 'ref_num'